In [ ]:
import sys
import re
from random import shuffle

from sklearn import svm
from sklearn.metrics import classification_report

from nltk.stem import PorterStemmer
from nltk.classify.scikitlearn import SklearnClassifier
import nltk.corpus
from nltk import ngrams, FreqDist

In [ ]:
from svmutil import *

In [ ]:
def get_znacilke(komentarji, vse_besede):    
    '''
    Metoda vrne značilke, ki se pojavljajo v posameznem komentarju
    '''    
    urejene_besede = sorted(vse_besede)
    znacilke = []    
    for komentar in komentarji:
        word_map = {}
        for beseda in urejene_besede:
            word_map[beseda] = 0
        for del_komentarja in komentar:
            if(del_komentarja in word_map):
                word_map[del_komentarja] = 1
        values = list(word_map.values())
        znacilke.append(values)
    return znacilke

In [ ]:
def get_znacilke_oznake(komentarji, vse_besede):
    '''
    Metoda vrne slovar z tipi komentarji in značilkami, ki se pojavljajo v posameznem komentarju
    '''    
    urejene_besede = sorted(vse_besede)
    znacilke = []
    oznake = []    
    for komentar in komentarji:
        oznaka = 0
        word_map = {}
        for beseda in urejene_besede:
            word_map[beseda] = 0        
        if len(komentar) > 1:
            besedilo_komentarja = komentar[0]
            zaljivost_komentarja = komentar[1]
            for del_komentarja in besedilo_komentarja:
                word_map[del_komentarja] = 1
            values = list(word_map.values())
            znacilke.append(values)
            if zaljivost_komentarja == 0:
                oznaka = 0
            elif zaljivost_komentarja == 1:
                oznaka = 1
            oznake.append(oznaka)        
    return {'pojavitev_besed' : znacilke, 'oznake':oznake}

In [ ]:
def preuredi_besedilo(tekst):
    '''
    Metoda vrne prečiščen tekst
    '''
    tekst = tekst.lower()
    tekst = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','', tekst)
    tekst = re.sub('newline_token', '', tekst)
    tekst = re.sub('newline', '', tekst)
    tekst = re.sub('tokennewline', '', tekst)
    tekst = re.sub('tab_token', '', tekst)
    tekst = re.sub('``', '"', tekst)
    tekst = re.sub('`', '\'', tekst)
    tekst = re.sub('=', '', tekst)
    tekst = re.sub('\n', '', tekst)
    tekst = re.sub(':', '', tekst)
    tekst = re.sub('[\s]+', ' ', tekst)
    tekst = tekst.strip()
    tekst = tekst.strip('\'"')
    return tekst

In [ ]:
def get_posamezne_besede(komentar, stopwords):    
    '''
    Metoda odstrani presledke, končnice in vrne posamezne besede. 
    Odstranimo nepotrebne whitespace in spremenimo v male črke
    '''
    komentar = preuredi_besedilo(komentar)
    komentar = " ".join(re.split("[^a-zA-Z]*", komentar.lower())).strip()    
    stemmer = PorterStemmer()
    besede = [] 
    for beseda in komentar.split():
        if not beseda in stopwords:
            try:
                besede.append(stemmer.stem(beseda))
            except(IndexError):
                besede.append(beseda)
    return besede

In [ ]:
def get_ngram(komentar, min_n, max_n, stopwords, join=False):
    '''
    Metoda vrne max_n - min_n razlicnih tipove ngramov 
    '''
    cngrams = []
    besede = ' '.join(get_posamezne_besede(komentar, stopwords)).split()
    for i in range(min_n, max_n + 1):
        cngrams.append(ngrams(besede, i))
    if join: 
        return [e for ngram in cngrams for e in list(ngram)]
    else: 
        return cngrams

In [ ]:
def word2ngrams(text, n=3, exact=True):
    '''
    Vrne ngrame nad besedami
    '''
    return ["".join(j) for j in zip(*[text[i:] for i in range(n)])]

In [ ]:
def get_all_words(tekst, algoritem, stopwords):
    '''
    Vrne vse bsede v tekstu
    '''
    rdata = []
    for comment in tekst:
        comment = comment.split('\t')
        offens_type = int(comment[0])
        comment = comment[1]
        for e in algoritem(komentar=comment, stopwords=stopwords):
            rdata.append(e)
    return rdata

In [ ]:
def run(tekst, test_tekst, algoritem):
    stopwords = stopwords.words("english")
    komentarji = []
    besede = []
    for vrstica in tekst:
        vrstica = vrstica.split("\t")
        tip_zaljivosti = int(vrstica[0])
        komentar = vrstica[1]
        if algoritem == "BOW":
            besede_komentarja = get_posamezne_besede(komentar, stopwords)
        elif algoritem == "NGRAM":
            besede_komentarja = get_ngram(komentar, 1, 3, stopwords)
        komentarji.append((besede_komentarja, tip_zaljivosti))
        for beseda in besede_komentarja:
            besede.append(beseda)

    znacilke_oznake = get_znacilke_oznake(komentarji, besede)    
    problem = svm_problem(znacilke_oznake['oznake'], znacilke_oznake['pojavitev_besed'])
    param = svm_parameter('-s 1 -b 1')
    param.kernel_type = LINEAR
    ucenec = svm_train(problem, param)
    
    test_besede = []
    test_zaljivost = []
    for vrstica in test_tekst:
        vrstica = vrstica.split("\t")
        tip_zaljivosti = int(vrstica[0])
        komentar = vrstica[1]        
        if algoritem == "BOW":
            test_besede_komentarja = get_posamezne_besede(komentar, stopwords)
        elif algoritem == "NGRAM":
            test_besede_komentarja = get_ngram(komentar, 1, 3, stopwords)        
        test_besede.append(test_besede_komentarja)
        test_zaljivost.append(tip_zaljivosti)

    print ("\n" + algoritem)
    test_znacilke = get_znacilke(test_besede, besede)
    predvidevana_zaljivost, a, b = svm_predict(test_zaljivost, test_znacilke, ucenec, options = "-b 1")
    print(classification_report(test_zaljivost, predvidevana_zaljivost))

In [ ]:
def run(tekst, test_tekst, algoritem, model):
    

In [ ]:
TRAIN_SET_PER = 0.7
CORP_SIZE = 3000

def main():
    # Pridobitev vseh podatkov
    datoteka = open('comments.txt' encoding="ISO-8859-1")
    tekst = datoteka.readlines()[1:CORP_SIZE]
    datoteka.close()
    
    shuffle(tekst)
    
    st_komentarjev = len(tekst)
    deli = st_komentarjev * TRAIN_SET_PER
    
    # Pridobitev učnih podatkov
    # ucna_datoteka = open('data/comments_sample.txt')
    # ucna_tekst = ucna_datoteka.readlines()[1:]
    # ucna_datoteka.close()
    
    ucna_tekst = tekst[1:5000]
    
    # Pridobitev testnih podatkov
    # test_datoteka = open("data/test_comments.txt")
    # test_tekst = test_datoteka.readlines()[1:]
    # test_datoteka.close()
    
    test_tekst = tekst[5000:6000]
    
    run(ucna_tekst, test_tekst, "BOW")
    run(ucna_tekst, test_tekst, "NGRAM")

In [ ]:
main()

Kako učimo na učni množici:
    1. Preberemo učno datoteko s komentarji (tip komentarja in tekst)
    2. Pridobimo stopworde (besede brez pomena)
    3. Vsako vrstico razdelimo na tip in sam komentar:
        1. Komentar uredimo (odstranimo odvečne znake, odstranimo končnice, pripravimo na procesiranje)
        2. Pridobimo besede v posameznem komentarju, dodamo jih v 'slovar besed'
    4. Slovar besed in komentarje obdelamo:
        1. V slovarju za vsak komentar shranimo tip komentarja in besede ki se oz. 
        se ne pojavljajo v komentarju
        2. Besede, ki se pojavijo v slovarju in v komentarju so označene z 1, ostale z 0.

Kako testiramo testno množico:
    1. Preberemo testno datoteko s komentarji (top komentarja in tekst)
    2. Pridobimo stopworde (besede brez pomena)
    3. Vsako vrstico razbijemo na tip in sam komentar:
        1. Tip si shranimo za kasnejše preverjanje uspešnosti klasifikacije komentarja
        2. Komentar uredimo (odstranimo odvečne znake, odstranimo končnice, pripravimo na procesiranje)
        3. Pridobimo besede v posameznem komentarju, dodamo jih v 'slovar testnih besed'
    4. Slovar besed in testnih komentarjev obdelamo:
        1. Pridobimo informacije o besedah ki se oz. se ne pojavljaju v komentarju
        2. Besede, ki se pojavijo v slovarju in komentarju so označene z 1, ostale z 0.
    5. Komentarjem na podlagi pojavljanja besed določimo tip (svm_predict)
    6. Izračunamo verjetnost (pravilno določeni tipi / vsi določeni tipi)*100
   
    
        